# Mini Projet de théorie des graphes en utilisant A*

# les bibliothèques utilisées

In [2]:
import osmnx as ox
import networkx as nx
import geopy
from geopy.geocoders import Nominatim
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from tkinter import Tk, Label, Button, StringVar, OptionMenu
import os
import webbrowser
import folium

# Matrice

In [3]:
place_name = "Fes, Morocco"
graph1 = ox.graph_from_place(place_name, network_type='all')

# Itérez sur les nœuds du graphe et imprimez les coordonnées de chaque point
def matrice_adj(max_points_to_print = 100,points_printed = 0,M=[],graph=graph1):
    for node, data in graph.nodes(data=True):
        if points_printed < max_points_to_print:
            M.append((data['y'],data['x']))
            points_printed += 1
        else:
            break
    return M

# Dictionnaire

In [4]:
geolocator = Nominatim(user_agent="my_geocoder")

coordinates = matrice_adj()

def loca(coordinate=coordinates,locations_dict={}):
    for coord in coordinate:
        location = geolocator.reverse(coord, language="fr")
        locations_dict[location.address] = coord
    return locations_dict


# Recherche de plus court chemin avec A*

In [ ]:
class ShortestPathApp:
    def __init__(self, master):
        self.master = master
        master.title("plus court chemin dans la ville Fès")

        # Define locations_dict before using it in __init__
        self.locations_dict = loca()
        self.label_start = Label(master, text="Point de départ:")
        self.label_start.grid(row=0, column=0, sticky="w", padx=10, pady=5)

        self.start_location_var = StringVar()
        self.entry_start = OptionMenu(master, self.start_location_var, *self.get_predefined_places())
        self.entry_start.grid(row=0, column=1, columnspan=2, pady=5)

        self.label_end = Label(master, text="Point d'arrivée:")
        self.label_end.grid(row=1, column=0, sticky="w", padx=10, pady=5)

        self.end_location_var = StringVar()
        self.entry_end = OptionMenu(master, self.end_location_var, *self.get_predefined_places())
        self.entry_end.grid(row=1, column=1, columnspan=2, pady=5)

        self.button_find_path = Button(master, text="Trouver le chemin le plus court", command=self.find_shortest_path)
        self.button_find_path.grid(row=2, column=0, columnspan=3, pady=10)
        
    def a_star(self, graph, start_node, end_node, heuristic, cost):
        open_list = [(0, start_node)]
        closed_set = set()
        came_from = {}
        g_score = {node: float('inf') for node in graph.nodes}
        g_score[start_node] = 0
        f_score = {node: float('inf') for node in graph.nodes}
        f_score[start_node] = heuristic(start_node, end_node)
        
        while open_list:
            current_f, current_node = min(open_list)
            if current_node == end_node:
                path = [current_node]
                while current_node in came_from:
                    current_node = came_from[current_node]
                    path.append(current_node)
                return path[::-1]
            
            open_list.remove((current_f, current_node))
            closed_set.add(current_node)
            
            for neighbor in graph.neighbors(current_node):
                if neighbor in closed_set:
                    continue
                edge_data = graph.get_edge_data(current_node, neighbor)
                if edge_data:
                    edge_length = edge_data.get('length', 1)  # Change 'length' to the correct key in your graph
                    tentative_g_score = g_score[current_node] + cost(current_node, neighbor, edge_length)
                    if tentative_g_score < g_score[neighbor]:
                        came_from[neighbor] = current_node
                        g_score[neighbor] = tentative_g_score
                        f_score[neighbor] = tentative_g_score + heuristic(neighbor, end_node)
                        if (f_score[neighbor], neighbor) not in open_list:
                            open_list.append((f_score[neighbor], neighbor))
        return None
    
    def find_shortest_path(self):
        start_location = self.start_location_var.get()
        end_location = self.end_location_var.get()
        start_coordinates = self.locations_dict.get(start_location)
        end_coordinates = self.locations_dict.get(end_location)

        if start_coordinates is not None and end_coordinates is not None:
            graph = ox.graph_from_place("Fes, Morocco", network_type='all')

            start_node = ox.distance.nearest_nodes(graph, start_coordinates[1], start_coordinates[0])
            end_node = ox.distance.nearest_nodes(graph, end_coordinates[1], end_coordinates[0])
            heuristic = lambda u, v: ox.distance.euclidean(graph.nodes[u]['y'], graph.nodes[u]['x'], graph.nodes[v]['y'], graph.nodes[v]['x'])

            # Utiliser votre propre implémentation de A*
            shortest_path = self.a_star(graph, start_node, end_node, heuristic, cost=lambda u, v, length: 1)

            # Create a Folium map centered around Fes
            m = folium.Map(location=[34.0181, -5.0077], zoom_start=13, control_scale=True)

            # Plot the shortest path
            path_coordinates = [[graph.nodes[node]['y'], graph.nodes[node]['x']] for node in shortest_path]
            folium.PolyLine(locations=path_coordinates, color='blue', weight=5, opacity=0.8).add_to(m)

            # Plot start and end locations
            folium.Marker(location=[start_coordinates[1], start_coordinates[0]], icon=folium.Icon(color='green')).add_to(m)
            folium.Marker(location=[end_coordinates[1], end_coordinates[0]], icon=folium.Icon(color='red')).add_to(m)

            # Display the map
            m.save('shortest_path_map.html')
            webbrowser.open('shortest_path_map.html')
    def get_predefined_places(self):
        return list(self.locations_dict.keys())
if __name__ == "__main__":
    root = Tk()
    app = ShortestPathApp(root)
    root.mainloop()
